# Getting Started with Python

## Import Statements

We're importing:
* [numpy](https://numpy.org/)
* [pandas](https://pandas.pydata.org/)
* [matplotlib.pyplot](https://matplotlib.org/stable/api/index.html)

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
# This allows us to create interactive plots with matplotlib
%matplotlib inline

print('Imports Successful')

## Importing Data

The original CBC dataset from 2018 came as three separate text (.csv) files.  Often times, we need to merge or **Join** information from different places.  First we need to read the .csv files with Pandas using the [.read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) function.
* This will load the files as "DataFrames"
    * We can set the header, automatically interpret dates, an set our table indexes


### incident.csv

Contains postal code of the incident, province, municipality, along with the date and incident ID


In [ ]:
incident = pd.read_csv(
    'data/incident.csv',
    delimiter = ',',
    header = 0,
    parse_dates=['date'],
    index_col=['id_incident']
)
print('incident.csv')
print(incident)

### victim.csv

Contains information on the victms including age, race, etc.

In [ ]:

victim = pd.read_csv(
    'data/victim.csv',
    delimiter = ',',
    header = 0, 
    index_col=['id_incident']
)
print('victim.csv')
victim.head()

### police.csv

Contaisn information about the police department and oficer involved

This import gives an error?  Any ideas why?

In [ ]:

police = pd.read_csv(
    'data police.csv',
    delimiter = ',',
    header = 0, 
    index_col=['id_incident']
)
police

## Joining Data

The three files share the same index **id_incident**.  This unique identifier can be used combine the three Dataframes into one.
* We can only join two dataframes at a time, but we can string together commands in one line.

Then we can change the index to the date using [set_index](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_index.html).

We can print the columns

In [ ]:
# Join the incident and the victim records then join police to that.
Join = incident.join(victim).join(police)

PID_Canada = Join.set_index('date',drop = True)

# .columns will give us all the column names (aka "headers")
print(PID_Canada.columns)

# .count() gives us the number of non "null" (missing) values for each column.
PID_Canada.count()


# Inspect at the data

Pandas has lots of functions we can use to inspect different variables.  One is [.describe()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html)
* It will calculate statistics for numeric data
* It is gives less information for text data
    * If we can list all unique text values with .unique()



In [ ]:
print('Descriptive Statistics: Age\n')
print(PID_Canada['age'].describe())
print()
print('Descriptive Statistics: Prov\n')
print(PID_Canada['prov'].describe())
print()
print('Unique Values: Prov\n')
print(PID_Canada['prov'].unique())

## Working with Dates

We can [resample](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html) by a specific intervals (e.g. "Y": Yearly, "M", Monthly,' etc.).  Resample will aggregate the data over a given time interval, .count() specifies how to aggregate.
* *Note* this only works when you have a date set as an index.

In [ ]:
Yearly = PID_Canada.resample('Y').count()


## The linregress() function calcualtes a linear regression between the year and the number of killings
LR = stats.linregress(Yearly.index.year,Yearly['id_victim'])

print('Linear Regression: Victims per Year')
print()
print(LR)

print()

Yearly

# Plotting

We can use [matplotlib.pyplot](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.html#module-matplotlib.pyplot) (plt) to make scatter bar charts and line graphs.

In [ ]:
# We can create a figure size 5x4
plt.figure(figsize=(6,5))

# .scatter() allows us to plot points
plt.bar(Yearly.index.year.astype(int),
            Yearly['id_victim'],
           color='red',
           edgecolor='k')

# .plots() allows us to plot lines
plt.plot(Yearly.index.year.astype(int),
         Yearly.index.year*LR[0]+LR[1],
         color='k',
         label='Increasing Trend: '+str(int(LR[0]))+' victim / year')

# We can set some specifics here.
plt.xticks(Yearly.index.year.astype(int),Yearly.index.year.astype(int),rotation=75)
plt.title('Police Killings by Year in Canada',loc='left')
plt.ylabel('Number of Victims')
plt.grid(axis='y')

# Calling .legend() will display all entites we set a label for
plt.legend()

# .tight_layout() allows us to make sure things fit nicely
plt.tight_layout()


## Aggregating

The [groupby](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html) function allows us conduct specific queries like:
### Were the victims armed?

In [ ]:
# .groupby() accepts one or more records to aggregate by
# .count() tells us how to aggregate
Armed = PID_Canada.groupby(['armed_type']).count()

Armed['id_victim'].sort_values()

## Editing data
* We can create a [dictionary](https://www.w3schools.com/python/python_dictionaries.asp) define the replacements we want to make
* We can use some of pandas special functions to query and manipulate our data
    * The [.loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) function allows us to search for records
    * The [.replace](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html) function lets us replace values

In [ ]:
# Dictionaries use keys (eg. 'Vehicle') an values (eg. 'Other weapon')
# They let us quickly look up values by a key
replace_dict = {
    'Air gun, replica gun':'Other weapons',
    'Bat, club, other swinging object':'Other weapons',
    'Vehicle':'Other weapons',
    'Knife, axe, other cutting instruments':'Knife',
    'Unknown':'None'
          }

# We can loop through te keys in the dictionary and use them to replace the disired values
# .loc[] is a search command that allows us to perform specific querries
# we can use it in combination with an equal sign (=) to replace values for a given column(s)
for r in replace_dict.keys():
    PID_Canada.loc[PID_Canada['armed_type']==r,'armed_type']=replace_dict[r]
    
Armed['id_victim'].sort_values()

## Pie Charts

In [ ]:
# Just making the same pic graph again
plt.figure(figsize=(6,6))
Armed = PID_Canada.groupby(['armed_type']).count()
plt.pie(
    Armed['id_victim'],
    labels=Armed.index,
    textprops={'fontsize': 12},
    autopct='%1.1f%%',
    wedgeprops={"edgecolor":"k",'linewidth': 2, 'linestyle': 'dashed'}
)
plt.title('Was the Victim Armed?', fontsize=14)
plt.tight_layout()

## More complex queries

Which police departments in Canada are most likely to kill an unarmed person?  Groupby can also accept multiple inputs.  

In [ ]:
# Groupby allows us to search fo multiple records
PoliceForce = PID_Canada.groupby(['Department','armed_type']).count().sort_values(by = 'id_victim',ascending=False)
PoliceForce.head(10)

## Formatting the result

Most columns are duplicated, we only need to keep one ('id_victim')

In [ ]:
Temp=PoliceForce['id_victim']
print(Temp)
print()
PoliceForce_by_ArmedType = Temp.unstack().sort_values(by='None')#,ascending=False)
print(PoliceForce_by_ArmedType)
print()
Top5_Unnarmed_Killings = PoliceForce_by_ArmedType.dropna()[-5:]
print(Top5_Unnarmed_Killings['None'])
print()


fig,ax= plt.subplots(1,1,figsize=(8,6))

ax.barh(Top5_Unnarmed_Killings.index,
        Top5_Unnarmed_Killings['None'],
        facecolor='#FF0000',
        edgecolor='black'
       )

ax.set_title('Unarmed Victims by Police Department')
plt.grid()
plt.tight_layout()


## The Demographic Distribution

BIPOC are severely overrepresented.

In [ ]:
PID_Canada.groupby('race').count()['id_victim'].sort_values()

## Normalizing

Canada is predominately white, we have to scale each group by the size of their population to calculate a police killing rate
* We want calculate the Police Killing Rate per Million Residents per Year for White, Black, and Indigenous people.
* What should we use as the scale factor?
    * Hint the dataset spans the years 2000 to 2017
    

In [ ]:
Race=['Caucasian','Black','Indigenous']
Population=[25803368,1198545,1673780]

scale = 1e6/18

Count = PID_Canada.groupby('race')['id_victim'].count()

plt.figure(figsize=(8,6))

i=0
for race,population in (zip(Race,Population)):
    if race == 'Total Population':
        rate=((Count.sum()/population)*scale)
    else:
        rate=((Count[race]/population)*1e6/18)
    plt.barh(i,rate,color='#FF0000',edgecolor='black')
    i += 1
plt.yticks([0,1,2],Race)
plt.title('Police Killing Rates in Canada (2000-2017) CBC')
plt.xlabel('Killing/Year/Million People')
plt.tight_layout()
    
plt.grid()

## Saving Data

* We can save our data easily using the [.tocsv()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) function



In [ ]:
PID_Canada.to_csv('data/PID_Canada.csv')
print('Data Saved')
print(PID_Canada)